In [5]:
import pandas as pd
df = pd.DataFrame({"CLT_CCODE_DOC": ["100", "100", "102", "102","102"],\
    "ITEM_NUM": ["1", "2", "1","2","3"],\
    "DEBIT_CREDIT": ["50", "CA", "EW","40","54"],\
    "GL_ACCOUNT": ["50000", "56000", "65000","70000", "45000"],\
    "AMOUNT_TR": [45000,45000,5000,4000,10000],
    'DOC_NUM': ["100", "100", "102", "102","102"], 
    'CLIENT': ["110", "110", "110", "110","110"], 
    'COCODE': ["CA01", "CA01", "CA01", "CA01","CA01"], 
    'DOC_DATE': ["15/03/2022", "15/03/2022", "11/05/2022", "11/05/2022","11/05/2022"], 
    'POSTING_DATE': ["4565", "4565", "4565", "4565","4565"]})
df

,CLT_CCODE_DOC,ITEM_NUM,DEBIT_CREDIT,GL_ACCOUNT,AMOUNT_TR,DOC_NUM,CLIENT,COCODE,DOC_DATE,POSTING_DATE
0,100,1,50,50000,45000,100,110,CA01,15/03/2022,4565
1,100,2,CA,56000,45000,100,110,CA01,15/03/2022,4565
2,102,1,EW,65000,5000,102,110,CA01,11/05/2022,4565
3,102,2,40,70000,4000,102,110,CA01,11/05/2022,4565
4,102,3,54,45000,10000,102,110,CA01,11/05/2022,4565


In [6]:
def compress_trans_data(df,max_j_lines,BSEG_cols,BKPF_cols,CLT_CCODE_DOC):
    #merge lists to get list of all coilumns required
    columns_cmprs = BSEG_cols.copy()
    columns_cmprs.extend(CLT_CCODE_DOC)  #all columns
    #create new dataframe with unique doc (trans ID) and transaction data 
    df_cmprs = df[columns_cmprs]
    #get only items that are within max_j_lines 
    df_cmprs["ITEM_NUM"] = df_cmprs["ITEM_NUM"].astype('int')
    df_cmprs = df_cmprs.loc[df_cmprs["ITEM_NUM"] <= max_j_lines]
    df_cmprs["ITEM_NUM"] = df_cmprs["ITEM_NUM"].astype('str')
    #getting unique documents to list
    all_journals_ls = df_cmprs[CLT_CCODE_DOC[0]].astype(str).unique().tolist()
    for i in range(0, len(all_journals_ls)):
        #get document in form of dataframe
        df_je=df_cmprs.loc[df_cmprs[CLT_CCODE_DOC[0]]==all_journals_ls[i]]

        for col in BSEG_cols:
            #get list of each columns
            je_col_ls = df_je[col].astype(str).tolist() 
            #add those list as a new column
            df_cmprs.loc[df_cmprs[CLT_CCODE_DOC[0]]==all_journals_ls[i], f"{col}_cmprs"] = '|'.join(je_col_ls)
    #drop not required columns
    df_cmprs = df_cmprs.drop(BSEG_cols, axis=1)
    #drop duplicate lines, keep only first line
    df_cmprs = df_cmprs.drop_duplicates(subset = CLT_CCODE_DOC, keep = "first")
    
    #prepare main df to get header data
    columns_main = BKPF_cols.copy()
    columns_main.extend(CLT_CCODE_DOC)  #all columns for main df
    df_main = df[columns_main]
    #df_main = df_main.drop(BSEG_cols, axis=1)
    df_main = df_main.drop_duplicates(subset = CLT_CCODE_DOC, keep = "first")
    df_cmprs = df_cmprs.merge(df_main, on=CLT_CCODE_DOC, how='left')
    return df_cmprs


In [3]:
# Parameters
max_j_lines = 5
BSEG_cols = ["ITEM_NUM","DEBIT_CREDIT", "GL_ACCOUNT", "AMOUNT_TR"]  #columns to be compressed
BKPF_cols = ["CLIENT","COCODE","DOC_NUM","POSTING_DATE","DOC_DATE"] #header data
CLT_CCODE_DOC = ["CLT_CCODE_DOC"]   #only one unique column title allowed


# RUN COMPRESSION ALGO TRANSACTION DATA FEATURES
df_compressed = compress_trans_data(df,max_j_lines,BSEG_cols,BKPF_cols,CLT_CCODE_DOC)


<ipython-input-2-3c0023ba2f60>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cmprs["ITEM_NUM"] = df_cmprs["ITEM_NUM"].astype('int')
